# Imports and Setup

In [25]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from scipy import ndimage
import pickle
import copy
import random
import time

torch.set_printoptions(precision=3)
cuda = True if torch.cuda.is_available() else False
torch.cuda.is_available()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data Entry and Processing

In [4]:
# Transform image to tensor and normalize features from [0,255] to [0,1]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,),(0.5,),(0.5)),
                                ])

In [5]:
# Using CIFAR100
traindata = datasets.CIFAR100('./data', download=True, train=True, transform=transform)
testdata = datasets.CIFAR100('./data', download=True, train=False, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# Loaders that give 64 example batches
cifar_train_loader = torch.utils.data.DataLoader(traindata, batch_size=64, shuffle=True)
cifar_test_loader = torch.utils.data.DataLoader(testdata, batch_size=64, shuffle=True)

In [7]:
# Transform image to tensor and normalize features from [0,255] to [0,1]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,),(0.5,)),
                                ])

In [8]:
# Using MNIST
traindata = datasets.MNIST('./data', download=True, train=True, transform=transform)
testdata = datasets.MNIST('./data', download=True, train=False, transform=transform)

In [9]:
mnist_train_loader = torch.utils.data.DataLoader(traindata, batch_size=64, shuffle=True)
mnist_test_loader = torch.utils.data.DataLoader(testdata, batch_size=64, shuffle=True)


In [10]:
data, label = next(iter(mnist_train_loader))
print(data.size())
print(len(mnist_train_loader.dataset))
print(len(mnist_train_loader))
# for batch_idx, (data, target) in enumerate(mnist_train_loader):
#     print(batch_idx) #938
# print(mnist_train_loader)

torch.Size([64, 1, 28, 28])
60000
938


# Model

In [11]:
# Hyperparameters
log_interval = 10
num_classes = 100
torch.backends.cudnn.enabled = True
criterion = F.nll_loss

In [12]:
# load resnet 18 and change to fit problem dimensionality
resnet = models.resnet18()
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
resnet.fc = nn.Sequential(nn.Linear(512, num_classes), nn.LogSoftmax(dim=1))
optimizer = optim.Adam(resnet.parameters())
# optimizer.to(device)
resnet = resnet.to(device)
print(resnet)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
# Training method that saves batch updates
def train(model, epoch, loader, returnable=False, amnesiac=True):
  model.train()
  deltas = []
  if amnesiac:
    for _ in range(50):
        delta = {}
        for param_tensor in model.state_dict():
          if "weight" in param_tensor or "bias" in param_tensor:
              delta[param_tensor] = 0
        deltas.append(delta)
    before = {}
    for param_tensor in model.state_dict():
        if "weight" in param_tensor or "bias" in param_tensor:
            before[param_tensor] = model.state_dict()[param_tensor].detach().cpu()
  for batch_idx, (data, target) in enumerate(loader):
    optimizer.zero_grad()
    output = model(data.to(device))
    loss = criterion(output, target.to(device))
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0 and batch_idx < 500 and amnesiac:
      after = {}
      for param_tensor in model.state_dict():
        if "weight" in param_tensor or "bias" in param_tensor:
          after[param_tensor] = model.state_dict()[param_tensor].detach().cpu()
      for key in before:
        deltas[batch_idx // 10][key] = after[key] - before[key]
      for param_tensor in model.state_dict():
        if "weight" in param_tensor or "bias" in param_tensor:
          before[param_tensor] = model.state_dict()[param_tensor].detach().cpu()
    if batch_idx % log_interval == 0:
      print("\rEpoch: {} [{:6d}]\tLoss: {:.6f}".format(
          epoch, batch_idx*len(data),  loss.item()
      ), end="")
  return deltas

In [14]:
# Testing method
def test(model, loader, dname="Test set", printable=True):
  model.eval()
  test_loss = 0
  total = 0
  correct = 0
  with torch.no_grad():
    for data, target in loader:
      output = model(data.to(device))
      total += target.size()[0]
      test_loss += criterion(output, target.to(device)).item()
      _, pred = torch.topk(output, 1, dim=1, largest=True, sorted=True)
      for i, t in enumerate(target):
        if t in pred[i]:
            correct += 1
  test_loss /= len(loader.dataset)
  if printable:
    print('{}: Mean loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        dname, test_loss, correct, total, 
        100. * correct / total
        ))
  return 1. * correct / total

# Original Training

In [15]:
trainingepochs = 10
num_classes = 10
counter = 0
for param_tensor in resnet.state_dict():
    if "weight" in param_tensor or "bias" in param_tensor:
        counter+=1
print(counter)

62


In [16]:
# Train new model for n epochs, saving parameter updates for
# sensitive batches
trainstart = time.process_time()
deltas = []
for _ in range(50):
  delta = {}
  for param_tensor in resnet.state_dict():
    if "weight" in param_tensor or "bias" in param_tensor:
        delta[param_tensor] = 0
  deltas.append(delta)
for epoch in range(1, trainingepochs+1):
  starttime = time.process_time()
  # train(resnet, epoch, all_data_train_loader, returnable=False)
  batch = train(resnet, epoch, mnist_train_loader, returnable=True) 
  for i in range(50):
    for key in deltas[i]:
        deltas[i][key] = batch[i][key] + deltas[i][key]
  test(resnet, mnist_test_loader, dname="All data")
  print(f"Time taken: {time.process_time() - starttime}")
print(f"Time taken for training: {time.process_time() - trainstart}")

Epoch: 1 [ 59520]	Loss: 0.050236All data: Mean loss: 0.0015, Accuracy: 9699/10000 (97%)
Time taken: 25.284268421
Epoch: 2 [ 59520]	Loss: 0.018188All data: Mean loss: 0.0005, Accuracy: 9903/10000 (99%)
Time taken: 25.011437279
Epoch: 3 [ 59520]	Loss: 0.004249All data: Mean loss: 0.0006, Accuracy: 9877/10000 (99%)
Time taken: 25.445805335999992
Epoch: 4 [ 59520]	Loss: 0.020490All data: Mean loss: 0.0005, Accuracy: 9898/10000 (99%)
Time taken: 25.479876761
Epoch: 5 [ 59520]	Loss: 0.002465All data: Mean loss: 0.0004, Accuracy: 9916/10000 (99%)
Time taken: 25.269557981999995
Epoch: 6 [ 59520]	Loss: 0.004145All data: Mean loss: 0.0004, Accuracy: 9918/10000 (99%)
Time taken: 25.00831100800002
Epoch: 7 [ 59520]	Loss: 0.033309All data: Mean loss: 0.0005, Accuracy: 9907/10000 (99%)
Time taken: 24.880864277
Epoch: 8 [ 59520]	Loss: 0.001414All data: Mean loss: 0.0005, Accuracy: 9901/10000 (99%)
Time taken: 25.021607434000003
Epoch: 9 [ 59520]	Loss: 0.014352All data: Mean loss: 0.0005, Accuracy: 99

In [17]:
with open('deltas.pickle', 'wb') as handle:
    pickle.dump(deltas, handle)

In [18]:
# # Train new model for n epochs, saving parameter updates for
# # ignoring 
# deltas = []
# for _ in range(50):
#   delta = {}
#   for param_tensor in resnet.state_dict():
#     if "weight" in param_tensor or "bias" in param_tensor:
#         delta[param_tensor] = 0
#   deltas.append(delta)
# for epoch in range(1, trainingepochs+1):
#   starttime = time.process_time()
#   # train(resnet, epoch, all_data_train_loader, returnable=False)
#   batch = train(resnet, epoch, mnist_train_loader, returnable=True) 
#   for i in range(50):
#     for key in deltas[i]:
#         deltas[i][key] = batch[i][key] + deltas[i][key]
#   test(resnet, mnist_test_loader, dname="All data")
#   print(f"Time taken: {time.process_time() - starttime}")

In [19]:
path = F"resnet/selective_mnist.pt"
torch.save({
            'model_state_dict': resnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, path)

In [20]:
path = F"resnet/selective_mnist.pt"
checkpoint = torch.load(path)
resnet.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [23]:
accuracy = [[],[],[],[],[],[],[],[],[],[]]
recordTime = [[],[],[],[],[],[],[],[],[],[]]
print(type(deltas))
# print(deltas)
# fileo = open(path, 'rb')
# deltas = pickle.load(fileo)
# fileo.close()
b = []
with open('deltas.pickle', 'rb') as handle:
    b = pickle.load(handle)
print(len(b))
print(len(deltas))
import os
print(os.path.getsize('deltas.pickle')/1073741824, ' GB')
# import sys
# print(sys.getsizeof(deltas)/1073741824)


<class 'list'>
50
50
2.0910925837233663  GB


In [26]:
# Begin amnesiac unlearning process, evaluating
# model accuracy as batches are removed
# 1 iteration = 1 batch
for j in range(10):
    random.shuffle(deltas)
    resnet.load_state_dict(checkpoint['model_state_dict'])
    for i in range(50):
        unlearnItStart = time.process_time()
        print(f"\riteration {j},{i}", end="")
        const = 1
        with torch.no_grad():
            state = resnet.state_dict()
            for param_tensor in state:
                if "weight" in param_tensor or "bias" in param_tensor:
                  state[param_tensor] = state[param_tensor] - const*deltas[i][param_tensor].cuda()
        resnet.load_state_dict(state)
        accuracy[j].append(test(resnet, mnist_test_loader, dname="All data"))
        end = time.process_time() - unlearnItStart
        recordTime[j].append(end)
        print(f"Time taken for unlearning: {end}")

iteration 0,0All data: Mean loss: 0.0009, Accuracy: 9832/10000 (98%)
Time taken for unlearning: 1.927261427000019
iteration 0,1All data: Mean loss: 0.0042, Accuracy: 9212/10000 (92%)
Time taken for unlearning: 1.8783095129999765
iteration 0,2All data: Mean loss: 0.0091, Accuracy: 8329/10000 (83%)
Time taken for unlearning: 1.8154999189999899
iteration 0,3All data: Mean loss: 0.0070, Accuracy: 8531/10000 (85%)
Time taken for unlearning: 1.7322403350000286
iteration 0,4All data: Mean loss: 0.0234, Accuracy: 6984/10000 (70%)
Time taken for unlearning: 1.7073080010000012
iteration 0,5All data: Mean loss: 0.0171, Accuracy: 7607/10000 (76%)
Time taken for unlearning: 1.7077986580000015
iteration 0,6All data: Mean loss: 0.0413, Accuracy: 5100/10000 (51%)
Time taken for unlearning: 1.6728555729999925
iteration 0,7All data: Mean loss: 0.0392, Accuracy: 5213/10000 (52%)
Time taken for unlearning: 1.745811869000022
iteration 0,8All data: Mean loss: 0.0392, Accuracy: 4827/10000 (48%)
Time taken fo

In [30]:
# path = F"selective_acc_mnist.pk"
# with open(path, 'w') as f:
#   for data in accuracy:
#     f.write(f"{data},")

# path = F"selective_time_mnist.pk"
# with open(path, 'w') as f:
#   for t in recordTime:
#     f.write(f"{t},")

In [31]:
f = open(F"selective_acc_mnist.pk", "wb")
pickle.dump(accuracy, f)
f.close()

In [35]:
f = open(F"selective_time_mnist.pk", "wb")
pickle.dump(recordTime, f)
f.close()

In [36]:
res = []
with open(F"selective_acc_mnist.pk", 'rb') as handle:
    res = pickle.load(handle)
print(len(res))
letime = []
with open(F"selective_time_mnist.pk", 'rb') as handle:
    letime = pickle.load(handle)
print(letime)

10
[[1.927261427000019, 1.8783095129999765, 1.8154999189999899, 1.7322403350000286, 1.7073080010000012, 1.7077986580000015, 1.6728555729999925, 1.745811869000022, 1.7602309519999721, 1.7049414490000458, 1.726317120000033, 1.707863331999988, 1.7589156189999926, 1.8727932900000042, 1.805004690999965, 1.6953128849999644, 1.7213650979999784, 1.7146969830000103, 1.6927118830000154, 1.7029503200000136, 1.6846932279999578, 1.6585565029999998, 1.6550000120000163, 1.6419752630000062, 1.7439204730000029, 1.6631311269999856, 1.639314225000021, 1.683530117000032, 1.66640583100002, 1.657054603000006, 1.7837315829999625, 1.7339133589999847, 1.8600033810000127, 1.6769398950000323, 1.685409667999977, 1.6897914520000086, 1.6906056939999985, 1.6606852119999758, 1.6927201130000071, 1.6754764610000166, 1.640058263999947, 1.724947051000015, 1.6873466719999897, 1.665097559000003, 1.6351508240000499, 1.649852758999998, 1.6654265770000052, 1.7072638359999814, 1.7617016969999781, 1.7842362620000358], [1.705847

In [11]:
dat = [0.9102, 0.8092, 0.777, 0.7303, 0.509, 0.4513, 0.4546, 0.5291, 0.4981, 0.474, 0.4847, 0.4434, 0.4718, 0.4838, 0.4531, 0.4352, 0.4201, 0.3791, 0.3653, 0.4171, 0.5038, 0.4068, 0.4163, 0.335, 0.2184, 0.2172, 0.3052, 0.3305, 0.334, 0.2979, 0.2434, 0.238, 0.2184, 0.2256, 0.2024, 0.2013, 0.1143, 0.1135, 0.1135, 0.1135, 0.1135, 0.1135, 0.1136, 0.1195, 0.1135, 0.1135, 0.1135, 0.1135, 0.1307, 0.1135]
print(len(dat))

50
